In [37]:

from openai import OpenAI

client = OpenAI(
    api_key="sk-5a569575d41b47b0bc9991d59e8fd3ab",
    base_url="https://api.deepseek.com"
)

# 定义 tools（函数/工具列表）
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "获取给定地点的天气",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市，比如杭州，北京，上海",
                    }
                },
                "required": ["location"],
            },
        },
    }
]

messages=[
    {"role": "system", "content": "你是一个很有用的 AI"},
    {"role": "user", "content": "今天北京的天气是什么?"},
]
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    tools=tools,  # 传入 tools 参数
    tool_choice="auto",  # 可选：控制是否强制调用某个工具
    stream=False,
)

print(response.choices[0].message)

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_182a32f6-8c90-40ae-9afe-1ef234ff5add', function=Function(arguments='{"location":"北京"}', name='get_current_weather'), type='function', index=0)])


In [52]:
messages=[
    {"role": "system", "content": "你是一个很有用的 AI"},
    {"role": "user", "content": "今天北京的天气是什么?"},
]

In [39]:
def get_current_weather(location):
    # 模拟一个天气查询的结果
    weather_data = {
        "location": location,
        "temperature": "25°C",
        "condition": "晴天"
    }
    return weather_data

In [40]:
print(response.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_0_182a32f6-8c90-40ae-9afe-1ef234ff5add', function=Function(arguments='{"location":"北京"}', name='get_current_weather'), type='function', index=0)]


In [41]:
## 执行function call
import json
tool_call = response.choices[0].message.tool_calls[0]
tool_name = tool_call.function.name
tool_args = json.loads(tool_call.function.arguments)
print(tool_name, tool_args)

get_current_weather {'location': '北京'}


In [43]:
function_call_result = get_current_weather(tool_args["location"])
print(function_call_result)

{'location': '北京', 'temperature': '25°C', 'condition': '晴天'}


In [44]:
print(tool_call.id)

call_0_182a32f6-8c90-40ae-9afe-1ef234ff5add


In [53]:
# 将function call的结果给LLM
messages.append(response.choices[0].message)
messages.append({
    "role": "tool",
    "content": json.dumps(function_call_result, ensure_ascii=False),
    "tool_call_id": tool_call.id
})

In [54]:
messages

[{'role': 'system', 'content': '你是一个很有用的 AI'},
 {'role': 'user', 'content': '今天北京的天气是什么?'},
 ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_182a32f6-8c90-40ae-9afe-1ef234ff5add', function=Function(arguments='{"location":"北京"}', name='get_current_weather'), type='function', index=0)]),
 {'role': 'tool',
  'content': '{"location": "北京", "temperature": "25°C", "condition": "晴天"}',
  'tool_call_id': 'call_0_182a32f6-8c90-40ae-9afe-1ef234ff5add'}]

In [55]:
# 再次调用LLM
res = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
)
print(res)
print(res.choices[0].message.content)

ChatCompletion(id='dd5d7cc7-5dc3-4cea-8a73-3ae6991f7996', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='今天北京的天气是晴天，气温25°C，适合外出活动哦！', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1744898939, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_3d5141a69a_prod0225', usage=CompletionUsage(completion_tokens=16, prompt_tokens=58, total_tokens=74, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0), prompt_cache_hit_tokens=0, prompt_cache_miss_tokens=58))
今天北京的天气是晴天，气温25°C，适合外出活动哦！


: 